In [1]:
import sys, random, math, pygame
from pygame.locals import *
from math import sqrt,cos,sin,atan2
from RRT_includes import *

In [2]:
#constants
XDIM = 720
YDIM = 500
WINSIZE = [XDIM, YDIM]
NUMNODES = 5000
GOAL_RADIUS = 15
MIN_DISTANCE_TO_ADD = 1.0
GAME_LEVEL = 1

PI = math.pi
UP = 1
DOWN = 2
LEFT = 3
RIGHT = 4

# Adjustments needed to use the same formula for left and right turns
RIGHT_ADJUSTMENT = 0
LEFT_ADJUSTMENT = PI

#want a 45 degree turn Epsilon long. that means EPSILON/theta = R
EPSILON = PI/2 * 20
TURN_THETA = PI/2
TURN_RADIUS = EPSILON / (TURN_THETA)    
DECAY = 1.0

white = 255, 240, 200
gray = 60, 60, 80
black = 20, 20, 40
red = 255, 0, 0
blue = 0, 0, 255
green = 0, 255, 0
cyan = 0,255,255

# setup program variables
count = 0
rectObs = []

In [3]:
# Adds obstacles to the map
def init_obstacles(configNum):
    global rectObs
    rectObs = []
    if (configNum == 0):
        rectObs.append(pygame.Rect((XDIM / 2.0 - 50, YDIM / 2.0 - 100),(100,200)))
    if (configNum == 1):
        rectObs.append(pygame.Rect((40,10),(100,200)))
        rectObs.append(pygame.Rect((500,200),(500,200)))
    if (configNum == 2):
        rectObs.append(pygame.Rect((40,10),(100,200)))
    if (configNum == 3):
        rectObs.append(pygame.Rect((40,10),(100,200)))
    for rect in rectObs:
        pygame.draw.rect(screen, red, rect)
        
# Clears and resets the map
def reset():
    global count
    screen.fill(black)
    init_obstacles(GAME_LEVEL)
    count = 0

In [4]:
# Calculates the collision based on start/end points and what kind of command we use to get there
def collides(p, start_pos=[-1, -1], command=-1):
    if command == -1:
        p_coords = get_coords(p)
        for rect in rectObs:
            if rect.collidepoint(p_coords) == True:
                return True
    else:
        trajectory = []
        pad = 50
        x1, y1, theta1 = start_pos
        x2, y2, theta2 = p
        width = abs(max(x1, x2) - min(x1, x2)) + pad
        height = abs(max(y1, y2) - min(y1, y2)) + pad
        left_x, top_y = min(x1, x2) - pad, max(y1, y2) - pad
        
        if command == UP or command == DOWN:
            # Rect((left, top), (width, height))
            left_x = min(x1, x2) - pad
            top_y = max(y1, y2) - pad
            trajectory.append(  pygame.Rect((left_x, top_y), (width, height))  )
        
        elif command == LEFT or command == RIGHT:
            # Case1
            if (x1==x2) and (y1>y2):
                opposite_corner = [x1 - TURN_RADIUS, y2]
                left_x, top_y = opposite_corner
                trajectory.append(pygame.Rect((left_x-pad, top_y-pad), (width, height)))
            elif (x1==x2) and (y1<y2):
                opposite_corner = [x1 + TURN_RADIUS, y2]
                left_x, top_y = x1, y1
                trajectory.append(pygame.Rect((left_x-pad, top_y-pad), (width, height)))
            # Case2
            elif (x1>x2) and (y1==y2):
                opposite_corner = [x2, y1 - TURN_RADIUS]
                left_x, top_y = x1, opposite_corner[1]
                trajectory.append(pygame.Rect((left_x-pad, top_y-pad), (width, height)))
            elif (x1<x2) and (y1==y2):
                opposite_corner = [x2, y1 + TURN_RADIUS]
                left_x, top_y = x1, y1
                trajectory.append(pygame.Rect((left_x-pad, top_y-pad), (width, height)))
            # Case3
            elif (x1 != x2) and (y1 != y2):
                trajectory.append(pygame.Rect((left_x-pad, top_y-pad), (width, height)))
        
        for t in trajectory:
            for rect in rectObs:
                if t == rect:
                    return True  
    return False

In [5]:
# Turns (x, y, theta) to [x, y] ... tuple to array
def get_coords(state):
    return state[0], state[1]

# Gets center coordinates given a radius and a point
def point_to_center(pos, radius, ADJUSTMENT):
    x = pos[0]
    y = pos[1]
    theta = pos[2]
    delta_x = radius * sin(theta + ADJUSTMENT)
    delta_y = radius * cos(theta + ADJUSTMENT)
    return x + delta_x, y + delta_y

# Gets a point's coordinates given a radius, center, and point orientation
def center_to_point(center, radius, theta, ADJUSTMENT):
    x = center[0]
    y = center[1]
    delta_x = - radius * sin(theta + ADJUSTMENT)
    delta_y = - radius * cos(theta + ADJUSTMENT)
    return x + delta_x, y + delta_y, theta

# Gets the endpoint of an arc
def end_of_arc(pos, delta_theta, radius, ADJUSTMENT):
    if ADJUSTMENT == RIGHT_ADJUSTMENT:
        delta_theta = - delta_theta
    center = point_to_center(pos, radius, ADJUSTMENT)
    theta = pos[2] + delta_theta
    end_point = center_to_point(center, radius, theta, ADJUSTMENT)
    return end_point

# Gets [x, y, height, width] of the arc "rectangle" (always a square here)
def get_arc_rectangle(start, radius, ADJUSTMENT):
    center = point_to_center(start, radius, ADJUSTMENT)
    rec_x = center[0] - radius
    rec_y  = center[1] - radius
    return rec_x, rec_y, radius*2, radius*2

# Gets start and end angles given two points
def get_arc_angles(start, end, cmd):
    startAngle = start[2] + TURN_THETA
    endAngle = end[2] + TURN_THETA        
    if cmd == LEFT or cmd == LEFT+2:
        startAngle += LEFT_ADJUSTMENT
        endAngle += LEFT_ADJUSTMENT
    if startAngle > endAngle:
        endAngle, startAngle = startAngle, endAngle
    return startAngle, endAngle

In [6]:
# Helper function used to calculale the distance between two points
def dist(p1,p2):
    return sqrt((p1[0]-p2[0])*(p1[0]-p2[0])+(p1[1]-p2[1])*(p1[1]-p2[1]))

# Helper function to evaluate the result of an UP command
def up_helper(p1):
    return p1[0] + EPSILON*cos(p1[2]), p1[1] + EPSILON*sin(p1[2]), p1[2]

# Helper function to evaluate the result of an DOWN command
def down_helper(p1):
    return p1[0] - EPSILON*cos(p1[2]), p1[1] - EPSILON*sin(p1[2]), p1[2]

# Helper function to evaluate the result of an LEFT command
def left_helper(p1, case):   
    if case == LEFT:
        return end_of_arc(p1, TURN_THETA, TURN_RADIUS, LEFT_ADJUSTMENT)
    else:
        return end_of_arc(p1, TURN_THETA/3*2, TURN_RADIUS, LEFT_ADJUSTMENT)

# Helper function to evaluate the result of an RIGHT command
def right_helper(p1, case):
    if case == RIGHT:
        return end_of_arc(p1, TURN_THETA, TURN_RADIUS, RIGHT_ADJUSTMENT)
    else:
        return end_of_arc(p1, TURN_THETA/3*2, TURN_RADIUS, RIGHT_ADJUSTMENT)

# Two functions that work to pick a random point as a sub-goal
def get_random():
    return random.random()*XDIM, random.random()*YDIM

def get_random_clear():
    while True:
        p = get_random()
        noCollision = collides(p)
        if noCollision == False:
            return p

In [7]:
# Picks one of six commands that brings the car closest to the endpos using the helper functions above
def helper_step(start_pos,end_pos):
    no_collision_commands = []
    no_collision_positions = []
    up_end_pos, down_end_pos, left_end_pos, right_end_pos = (-1, -1), (-1, -1), (-1, -1), (-1, -1)
    global EPSILON
    global TURN_RADIUS
    global DECAY
    
    for i in range(0, 6):
        command = i + 1
        if command == UP:
            up_end_pos = up_helper(start_pos)
            if collides(up_end_pos, start_pos, command) == False:
                no_collision_commands.append(command)
                no_collision_positions.append(up_end_pos)
        if command == DOWN:
            down_end_pos = down_helper(start_pos)
            if collides(down_end_pos, start_pos, command) == False:
                no_collision_commands.append(command)
                no_collision_positions.append(down_end_pos)
        if command == LEFT:
            left_end_pos = left_helper(start_pos, command)
            if collides(left_end_pos, start_pos, command) == False:
                no_collision_commands.append(command)
                no_collision_positions.append(left_end_pos)
        if command == RIGHT:
            right_end_pos = right_helper(start_pos, command)
            if collides(right_end_pos, start_pos, command) == False:
                no_collision_commands.append(command)
                no_collision_positions.append(right_end_pos)
        if command == LEFT+2:
            left_end_pos = left_helper(start_pos, command)
            if collides(left_end_pos, start_pos, command) == False:
                no_collision_commands.append(command)
                no_collision_positions.append(left_end_pos)
        if command == RIGHT+2:
            right_end_pos = right_helper(start_pos, command)
            if collides(right_end_pos, start_pos, command) == False:
                no_collision_commands.append(command)
                no_collision_positions.append(right_end_pos)
    best_command, best_end_pos, best_dist = -1, start_pos, 99999
    for i in range(len(no_collision_commands)):
        d = dist(end_pos, no_collision_positions[i])
        if  d < best_dist:
            best_dist = d
            best_command = no_collision_commands[i]
            best_end_pos = no_collision_positions[i]
        #TODO: check performance on decay
        if dist(best_end_pos, end_pos) < dist(start_pos, best_end_pos)/2:
            EPSILON = EPSILON*DECAY
            TURN_RADIUS -= 0.001
            DECAY *= DECAY
    return best_command, best_end_pos

In [8]:
#initialize and prepare screen
pygame.init()
fpsClock = pygame.time.Clock()

screen = pygame.display.set_mode(WINSIZE)
pygame.display.set_caption('Rapidly Exploring Random Tree')

In [ ]:
# Calculates the path from a start and end position given either through parameters or by clicking on the map
def run(start_pos=[0,0,0], end_pos=[0,0,0], given_start_and_end=False):
    global count
    turn_flag = 0
    
    reset()
    
    if given_start_and_end == True:
        start_pos = tuple(start_pos)
        end_pos = tuple(end_pos)
        nodes = []
        if collides(start_pos) == False and collides(end_pos) == False:
            initPoseSet = True
            goalPoseSet = True
            initialPoint = Node(start_pos, None)
            goalPoint = Node(end_pos, None)
            pygame.draw.circle(screen, blue, get_coords(initialPoint.point), GOAL_RADIUS)
            pygame.draw.circle(screen, green, get_coords(goalPoint.point), GOAL_RADIUS) 
            
            nodes.append(initialPoint)
            fpsClock.tick(10)
            currentState = 'buildTree'   
    else:
        nodes = []
        initPoseSet = False
        initialPoint = Node(None, None)
        goalPoseSet = False
        goalPoint = Node(None, None)
        currentState = 'init'

    while True:
        if currentState == 'init':
            fpsClock.tick(10)
        elif currentState == 'goalFound':
            #traceback
            currNode = goalNode
            while currNode.parent != None:
                #traceback
                currNode = goalNode
                while currNode.parent != None:
                    start = currNode.parent.point
                    end = currNode.point
                    if start[2] - end[2] == 0:
                        pygame.draw.line(screen,cyan,get_coords(start),get_coords(end))
                    else:
                        if start[2] - end[2] > 0:
                            if start[2] - end[2] > TURN_THETA/3*2:
                                bestcmd = RIGHT
                            else:
                                bestcmd = RIGHT + 2
                            adj = RIGHT_ADJUSTMENT
                        else:
                            if start[2] - end[2] < TURN_THETA/3*2:
                                bestcmd = LEFT
                            else:
                                bestcmd = LEFT + 2
                            adj = LEFT_ADJUSTMENT
                        startAngle, endAngle = get_arc_angles(start, end, bestcmd)
                        rec_x, rec_y, height, width = get_arc_rectangle(start, TURN_RADIUS, adj)
                        pygame.draw.arc(screen, cyan, [rec_x, rec_y, height, width], startAngle, endAngle, 1)
                        pygame.display.update()
                    currNode = currNode.parent

                optimizePhase = True
            optimizePhase = True
        elif currentState == 'optimize':
            fpsClock.tick(0.5)
            pass
        elif currentState == 'buildTree':
            count = count+1
            if count < NUMNODES:
                foundNext = False
                while foundNext == False:
                    rand = get_random_clear()
                    parentNode = nodes[0]

                    for p in nodes: #find nearest vertex
                        if dist(p.point,rand) <= dist(parentNode.point,rand): #check to see if this vertex is closer than the previously selected closest
                            newCommand, newPoint = helper_step(p.point, rand) #step_from_to(p.point,rand)
                            
                            if collides(newPoint) == False: # check if a collision would occur with the newly selected vertex
                                parentNode = p #the new point is not in collision, so update this new vertex as the best
                                foundNext = True
                                
                # Draws either an arc or a line depending on the command
                bestcmd, newnode = helper_step(parentNode.point,rand)
                nodes.append(Node(newnode, parentNode, bestcmd))
                if bestcmd == UP or bestcmd == DOWN:
                    pygame.draw.line(screen,gray,get_coords(parentNode.point),get_coords(newnode))
                else:
                    if bestcmd == LEFT or bestcmd == LEFT+2:
                        adj = LEFT_ADJUSTMENT
                    else:
                        adj = RIGHT_ADJUSTMENT
                    startAngle, endAngle = get_arc_angles(parentNode.point, newnode, bestcmd)
                    rec_x, rec_y, height, width = get_arc_rectangle(parentNode.point, TURN_RADIUS, adj)
                    pygame.draw.arc(screen, gray, [rec_x, rec_y, height, width], startAngle, endAngle, 1)
                    pygame.display.update()
                    
                if point_circle_collision(newnode, goalPoint.point, GOAL_RADIUS):
                    currentState = 'goalFound'
                    goalNode = nodes[len(nodes)-1]

        if given_start_and_end == False:
        #handle events
            for e in pygame.event.get():
                if e.type == QUIT or (e.type == KEYUP and e.key == K_ESCAPE):
                    sys.exit("Exiting")
                if e.type == MOUSEBUTTONDOWN:
                    print('mouse down')
                    if currentState == 'init':
                        if initPoseSet == False:
                            nodes = []
                            if collides(e.pos) == False:
                                startState = tuple([e.pos[0], e.pos[1], 0])
                                print('initial pose set: '+str(e.pos))

                                initialPoint = Node(startState, None)
                                nodes.append(initialPoint) # Start in the center
                                initPoseSet = True
                                pygame.draw.circle(screen, blue, get_coords(initialPoint.point), GOAL_RADIUS)
                        elif goalPoseSet == False:
                            print('goal pose set: '+str(e.pos))
                            if collides(e.pos) == False:
                                goalState = tuple([e.pos[0], e.pos[1], 0])
                                goalPoint = Node(goalState,None)
                                goalPoseSet = True
                                pygame.draw.circle(screen, green, get_coords(goalPoint.point), GOAL_RADIUS)
                                currentState = 'buildTree'
                    else:
                        currentState = 'init'
                        initPoseSet = False
                        goalPoseSet = False
                        reset()

        pygame.display.update()
        fpsClock.tick(10000)

def main():
    run()
    
# if python says run, then we should run
if __name__ == '__main__':
    main()
    input("press Enter to quit")